# Natural Language Processing (NLP) 🌐

## Introduction
`Natural Language Processing (NLP)` applications enable computers to understand, interpret, and respond to human language in a meaningful way. These applications range from conversation and language translation to sentiment analysis and text summarization. This notebook demonstrates how to build a `chatbot` using the `Hugging Face Transformers` library. We will go through the steps of setting up the environment, preparing the data, building the pipelines, training the model, and evaluating the results.

## Install and Import Libraries

Here is a brief description of the required libraries:

- The `Transformers library by Hugging Face` is a powerful open-source framework that provides pre-trained NLP models for tasks like text classification, translation, summarization, and more. It simplifies the process of using state-of-the-art NLP models with minimal code.

- The `pipeline function` from the transformers library by Hugging Face is used to easily access pre-trained models for various NLP tasks. By calling pipeline, you can quickly load models for tasks like text generation, translation, question answering, and more.

In [1]:
# Install and update the necessary libraries
%pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 49.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import the required libraries
from transformers import pipeline

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Suppress warning messages such as non-critical log messages
from transformers.utils import logging
logging.set_verbosity_error()

## Data Preparation
In this section, we will define the input data and prepare it for model training.

### Define Input Text

In [3]:
# Define the input text
user_message = """
What are some fun activities I can do in the winter?
"""
# Uncomment the following line to print the input message
# print(user_message)

### Model Training
Here, we will build and train the chatbot model using the Transformers library.

#### Build the `chatbot` pipeline using 🤗 Transformers Library

- Define the conversation pipeline

>_Note_ : In the past, a dedicated `ConversationalPipeline` class has been used and the Conversation object was specific to the conversational task and able to manage multi-turn conversations. However, the ConversationalPipeline has now been deprecated. Actually, the `TextGenerationPipeline`, that does not include a dedicated "conversational" feature, is used. 

The [BlenderBot model](https://huggingface.co/facebook/blenderbot-400M-distill) has been selected for `text2text-generation`. 

In [4]:
# Load the pipeline for text2text-generation using the Blenderbot model
chatbot = pipeline(task="text2text-generation", model="facebook/blenderbot-400M-distill") #the pipeline is used as a high-level helper

##### Generate Responses

 With text2text-generation, it is necessary to pass a plain string as input, and the model will handle the conversation. 

In [5]:
# Generate text by providing the input to the pipeline
conversation = chatbot(user_message)

In [6]:
# Print the input and the generated response
print(f"Input: {user_message}")
print(f"Output: {conversation[0]['generated_text']}")

Input: 
What are some fun activities I can do in the winter?

Output:  I like snowboarding and skiing.  What do you like to do in winter?


- You can continue the conversation with the chatbot however, the chatbot may provide an unrelated response because it does not have memory of any prior conversations.

- To include prior conversations in the LLM's context, using text2text-generation, you should define a list to hold the conversation history and define the sequence of the interactions.

In [7]:
# Initialize the pipeline for text-to-text generation
chatbot = pipeline("text2text-generation", model="facebook/blenderbot-400M-distill")

# Define a list to hold the conversation history
conversation = []

def add_message(role, content):
    """Helper function to add a message to the conversation history."""
    conversation.append({"role": role, "content": content})

def get_conversation_context():
    """Combine previous messages into a single string."""
    return "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in conversation])

# First interaction
add_message("user", "What are some fun activities I can do in the winter?")
# Initialize the pipeline for text-to-text generation
response_1 = chatbot(get_conversation_context())[0]['generated_text']
add_message("bot", response_1)

# Print the first conversation turn
print(f"Input: {conversation[-2]['content']}")
print(f"Output: {conversation[-1]['content']}")

# Second interaction, using conversation history
add_message("user", "What else do you recommend?")
conversation_context = get_conversation_context()
response_2 = chatbot(conversation_context)[0]['generated_text']
add_message("bot", response_2)

# Print the updated conversation history and the new response
print("\n--- Conversation so far ---")
print(get_conversation_context())

Input: What are some fun activities I can do in the winter?
Output:  I'm not sure, but I'm sure you can find something fun to do in winter.

--- Conversation so far ---
User: What are some fun activities I can do in the winter?
Bot:  I'm not sure, but I'm sure you can find something fun to do in winter.
User: What else do you recommend?
Bot:  I like to go snowboarding and skiing. What do you like to do?


#### Evaluation
Evaluate the chatbot's responses and continue the conversation.

In [8]:
# Third interaction, adding a new user question and the bot's response
add_message("user", "What about to go schatting?")
conversation_context = get_conversation_context()
response_3 = chatbot(conversation_context)[0]['generated_text']
add_message("bot", response_3)

# Print the updated conversation history and the third response
print("\n--- Updated Conversation so far ---")
print(get_conversation_context())


--- Updated Conversation so far ---
User: What are some fun activities I can do in the winter?
Bot:  I'm not sure, but I'm sure you can find something fun to do in winter.
User: What else do you recommend?
Bot:  I like to go snowboarding and skiing. What do you like to do?
User: What about to go schatting?
Bot:  I don't think I've ever done that before, but it sounds like a lot of fun!


**Evaluation output** : Analyzing the above provided conversation, several evaluations can be performed regarding the bot's performance, including:  
- _linguistic_ (e.g.: Grammar and Syntax, Spelling Errors): bot's responses are grammatically correct and syntactically appropriate, which maintains a natural flow in the conversation. However, the user made a spelling mistake with "schatting" (likely meant to be "skating"), and the bot didn't identify or address this typo.
- _contextual_ (e.g.Misinterpretation of User Input, Context Relevance): lack of error correction and semantic understanding (Misinterpretation of User Input) 
- _user experience evaluations_ (e.g.: Engagement, Adaptability): bot maintains always a positive tone, its response misses an opportunity to engage with the activity or clarify the user's intent.  

## Conclusion
This notebook demonstrated how to build a chatbot using the `Hugging Face Transformers library`. and the `BlenderBot 400M-distill model` for `text2text-generation (tasks)`. However the selected models has several advantages and disadvantages that influence its output and performance. Infact, BlenderBot 400M-distill is able to generate coherent conversational responses and handling diverse inputs, making it suitable for resource-limited environments. However, its outdated knowledge, shallow grasp of complex topics, sensitivity to input phrasing, and tendency for generic responses are drawbacks. It suits lightweight conversational tasks, but GPT models may excel in creative reasoning.

## Extra practice  
- Try chatting with the model!
- Experiment with different models and parameters.
